In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints_DANN'
my_epoch = 500

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = cv2.imread(img_dir)
#         img = cv2.resize(img,(32,32))
#         X.append(img)
#     X = np.array(X)
#     return X

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L') # conver to grayscale images
        img = img.resize([32, 16])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(35, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=35)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=35)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=35)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=35)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=35)
db_images,db_labels = get_img_label('db', 'all', num_classes=35)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=35)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=35)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=35)

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(weather_images_t))
print(np.shape(weather_labels_t))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(base_images, base_labels,weather_images_t,weather_labels_t, epochs=my_epoch,verbose=1,batch_size = 32)
weather_model.score(base_images, base_labels)
weather_model.score(weather_images, weather_labels)
weather_model.save_weights(save_check_pt+ '/weather')

(8446, 512)
(8446, 35)
(762, 512)
(762, 35)
Epoch 1/500
286/286 [==============================] - 2s 3ms/step - loss: 2.7653 - acc: 0.2784 - disc_loss: 1.4179 - disc_acc: 0.4910
Epoch 2/500
286/286 [==============================] - 1s 3ms/step - loss: 2.1277 - acc: 0.4530 - disc_loss: 1.3794 - disc_acc: 0.5374
Epoch 3/500
286/286 [==============================] - 1s 3ms/step - loss: 1.6835 - acc: 0.6087 - disc_loss: 1.3785 - disc_acc: 0.5349
Epoch 4/500
286/286 [==============================] - 1s 3ms/step - loss: 1.4172 - acc: 0.6558 - disc_loss: 1.3810 - disc_acc: 0.5270
Epoch 5/500
286/286 [==============================] - 1s 3ms/step - loss: 1.2239 - acc: 0.6867 - disc_loss: 1.3730 - disc_acc: 0.5506
Epoch 6/500
286/286 [==============================] - 1s 3ms/step - loss: 1.0296 - acc: 0.7208 - disc_loss: 1.3780 - disc_acc: 0.5377
Epoch 7/500
286/286 [==============================] - 1s 3ms/step - loss: 0.9041 - acc: 0.7535 - disc_loss: 1.3777 - disc_acc: 0.5350
Epoch 8/500

Epoch 61/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0873 - acc: 0.9775 - disc_loss: 1.3776 - disc_acc: 0.5344
Epoch 62/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0751 - acc: 0.9816 - disc_loss: 1.3787 - disc_acc: 0.5339
Epoch 63/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0653 - acc: 0.9851 - disc_loss: 1.3804 - disc_acc: 0.5320
Epoch 64/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0579 - acc: 0.9862 - disc_loss: 1.3813 - disc_acc: 0.5308
Epoch 65/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0604 - acc: 0.9850 - disc_loss: 1.3787 - disc_acc: 0.5352
Epoch 66/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0717 - acc: 0.9821 - disc_loss: 1.3794 - disc_acc: 0.5331
Epoch 67/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0898 - acc: 0.9770 - disc_loss: 1.3787 - disc_acc: 0.5383
Epoch 68/500
286/286 [==========================

286/286 [==============================] - 1s 3ms/step - loss: 0.0391 - acc: 0.9893 - disc_loss: 1.3813 - disc_acc: 0.5333
Epoch 122/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0421 - acc: 0.9886 - disc_loss: 1.3773 - disc_acc: 0.5408
Epoch 123/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0591 - acc: 0.9843 - disc_loss: 1.3773 - disc_acc: 0.5399
Epoch 124/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0545 - acc: 0.9850 - disc_loss: 1.3755 - disc_acc: 0.5444
Epoch 125/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0402 - acc: 0.9882 - disc_loss: 1.3803 - disc_acc: 0.5348
Epoch 126/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0379 - acc: 0.9900 - disc_loss: 1.3819 - disc_acc: 0.5334
Epoch 127/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0410 - acc: 0.9876 - disc_loss: 1.3784 - disc_acc: 0.5424
Epoch 128/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0374 - acc: 0.9899 - disc_loss: 1.3773 - disc_acc: 0.5377
Epoch 181/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0438 - acc: 0.9890 - disc_loss: 1.3793 - disc_acc: 0.5313
Epoch 182/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0289 - acc: 0.9926 - disc_loss: 1.3795 - disc_acc: 0.5369
Epoch 183/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0270 - acc: 0.9930 - disc_loss: 1.3788 - disc_acc: 0.5369
Epoch 184/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0335 - acc: 0.9903 - disc_loss: 1.3821 - disc_acc: 0.5306
Epoch 185/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0423 - acc: 0.9888 - disc_loss: 1.3797 - disc_acc: 0.5312
Epoch 186/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0382 - acc: 0.9883 - disc_loss: 1.3806 - disc_acc: 0.5305
Epoch 187/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0512 - acc: 0.9862 - disc_loss: 1.3769 - disc_acc: 0.5367
Epoch 240/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0401 - acc: 0.9903 - disc_loss: 1.3744 - disc_acc: 0.5475
Epoch 241/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0313 - acc: 0.9908 - disc_loss: 1.3790 - disc_acc: 0.5393
Epoch 242/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0310 - acc: 0.9913 - disc_loss: 1.3784 - disc_acc: 0.5357
Epoch 243/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0315 - acc: 0.9916 - disc_loss: 1.3754 - disc_acc: 0.5415
Epoch 244/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0295 - acc: 0.9925 - disc_loss: 1.3743 - disc_acc: 0.5443
Epoch 245/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0296 - acc: 0.9921 - disc_loss: 1.3775 - disc_acc: 0.5340
Epoch 246/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0432 - acc: 0.9887 - disc_loss: 1.3765 - disc_acc: 0.5390
Epoch 299/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0307 - acc: 0.9907 - disc_loss: 1.3788 - disc_acc: 0.5321
Epoch 300/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0416 - acc: 0.9879 - disc_loss: 1.3749 - disc_acc: 0.5439
Epoch 301/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0338 - acc: 0.9909 - disc_loss: 1.3743 - disc_acc: 0.5392
Epoch 302/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0287 - acc: 0.9918 - disc_loss: 1.3751 - disc_acc: 0.5418
Epoch 303/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0358 - acc: 0.9905 - disc_loss: 1.3735 - disc_acc: 0.5397
Epoch 304/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0400 - acc: 0.9883 - disc_loss: 1.3735 - disc_acc: 0.5462
Epoch 305/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0508 - acc: 0.9853 - disc_loss: 1.3740 - disc_acc: 0.5447
Epoch 358/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0293 - acc: 0.9919 - disc_loss: 1.3727 - disc_acc: 0.5474
Epoch 359/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0258 - acc: 0.9921 - disc_loss: 1.3779 - disc_acc: 0.5302
Epoch 360/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0271 - acc: 0.9932 - disc_loss: 1.3812 - disc_acc: 0.5309
Epoch 361/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0284 - acc: 0.9919 - disc_loss: 1.3800 - disc_acc: 0.5360
Epoch 362/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0271 - acc: 0.9933 - disc_loss: 1.3767 - disc_acc: 0.5394
Epoch 363/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0317 - acc: 0.9918 - disc_loss: 1.3771 - disc_acc: 0.5328
Epoch 364/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0638 - acc: 0.9820 - disc_loss: 1.3762 - disc_acc: 0.5404
Epoch 417/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0347 - acc: 0.9894 - disc_loss: 1.3724 - disc_acc: 0.5478
Epoch 418/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0411 - acc: 0.9881 - disc_loss: 1.3735 - disc_acc: 0.5482
Epoch 419/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0320 - acc: 0.9914 - disc_loss: 1.3733 - disc_acc: 0.5447
Epoch 420/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0645 - acc: 0.9811 - disc_loss: 1.3708 - disc_acc: 0.5480
Epoch 421/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0428 - acc: 0.9882 - disc_loss: 1.3721 - disc_acc: 0.5446
Epoch 422/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0310 - acc: 0.9933 - disc_loss: 1.3755 - disc_acc: 0.5440
Epoch 423/500
286/286 [==============================] 

286/286 [==============================] - 1s 3ms/step - loss: 0.0331 - acc: 0.9916 - disc_loss: 1.3759 - disc_acc: 0.5412
Epoch 476/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0429 - acc: 0.9876 - disc_loss: 1.3753 - disc_acc: 0.5467
Epoch 477/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0271 - acc: 0.9923 - disc_loss: 1.3767 - disc_acc: 0.5383
Epoch 478/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0283 - acc: 0.9927 - disc_loss: 1.3779 - disc_acc: 0.5325
Epoch 479/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0371 - acc: 0.9887 - disc_loss: 1.3747 - disc_acc: 0.5447
Epoch 480/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0375 - acc: 0.9892 - disc_loss: 1.3743 - disc_acc: 0.5443
Epoch 481/500
286/286 [==============================] - 1s 3ms/step - loss: 0.0186 - acc: 0.9956 - disc_loss: 1.3771 - disc_acc: 0.5367
Epoch 482/500
286/286 [==============================] 

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(challenge_images_t))
print(np.shape(challenge_labels_t))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(base_images, base_labels,challenge_images_t,challenge_labels_t, epochs=my_epoch,verbose=1,batch_size = 32)
challenge_model.score(base_images, base_labels)
challenge_model.score(challenge_images, challenge_labels)
challenge_model.save_weights(save_check_pt+ '/challenge')

(8446, 512)
(8446, 35)
(109, 512)
(109, 35)
Epoch 1/500
266/266 [==============================] - 2s 4ms/step - loss: 2.8378 - acc: 0.2768 - disc_loss: 1.4400 - disc_acc: 0.4895
Epoch 2/500
266/266 [==============================] - 1s 3ms/step - loss: 2.2045 - acc: 0.4618 - disc_loss: 1.3766 - disc_acc: 0.5366
Epoch 3/500
266/266 [==============================] - 1s 3ms/step - loss: 1.8280 - acc: 0.5614 - disc_loss: 1.3698 - disc_acc: 0.5555
Epoch 4/500
266/266 [==============================] - 1s 3ms/step - loss: 1.5199 - acc: 0.6210 - disc_loss: 1.3709 - disc_acc: 0.5516
Epoch 5/500
266/266 [==============================] - 1s 3ms/step - loss: 1.3050 - acc: 0.6582 - disc_loss: 1.3681 - disc_acc: 0.5514
Epoch 6/500
266/266 [==============================] - 1s 3ms/step - loss: 1.1440 - acc: 0.6880 - disc_loss: 1.3703 - disc_acc: 0.5504
Epoch 7/500
266/266 [==============================] - 1s 3ms/step - loss: 1.0183 - acc: 0.7178 - disc_loss: 1.3708 - disc_acc: 0.5613
Epoch 8/500

Epoch 61/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0996 - acc: 0.9754 - disc_loss: 1.3739 - disc_acc: 0.5440
Epoch 62/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0991 - acc: 0.9738 - disc_loss: 1.3696 - disc_acc: 0.5574
Epoch 63/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0956 - acc: 0.9766 - disc_loss: 1.3708 - disc_acc: 0.5576
Epoch 64/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0832 - acc: 0.9809 - disc_loss: 1.3799 - disc_acc: 0.5284
Epoch 65/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0863 - acc: 0.9765 - disc_loss: 1.3727 - disc_acc: 0.5453
Epoch 66/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0860 - acc: 0.9787 - disc_loss: 1.3717 - disc_acc: 0.5516
Epoch 67/500
266/266 [==============================] - 1s 3ms/step - loss: 0.1166 - acc: 0.9695 - disc_loss: 1.3673 - disc_acc: 0.5592
Epoch 68/500
266/266 [==========================

266/266 [==============================] - 1s 3ms/step - loss: 0.0525 - acc: 0.9865 - disc_loss: 1.3753 - disc_acc: 0.5365
Epoch 122/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0551 - acc: 0.9854 - disc_loss: 1.3801 - disc_acc: 0.5235
Epoch 123/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0633 - acc: 0.9842 - disc_loss: 1.3659 - disc_acc: 0.5658
Epoch 124/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0438 - acc: 0.9892 - disc_loss: 1.3744 - disc_acc: 0.5494
Epoch 125/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0557 - acc: 0.9871 - disc_loss: 1.3754 - disc_acc: 0.5480
Epoch 126/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0921 - acc: 0.9749 - disc_loss: 1.3648 - disc_acc: 0.5601
Epoch 127/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0704 - acc: 0.9814 - disc_loss: 1.3655 - disc_acc: 0.5541
Epoch 128/500
266/266 [==============================] 

266/266 [==============================] - 1s 3ms/step - loss: 0.0426 - acc: 0.9888 - disc_loss: 1.3793 - disc_acc: 0.5288
Epoch 181/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0477 - acc: 0.9871 - disc_loss: 1.3721 - disc_acc: 0.5360
Epoch 182/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0407 - acc: 0.9886 - disc_loss: 1.3700 - disc_acc: 0.5526
Epoch 183/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0403 - acc: 0.9893 - disc_loss: 1.3724 - disc_acc: 0.5427
Epoch 184/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0505 - acc: 0.9832 - disc_loss: 1.3704 - disc_acc: 0.5385
Epoch 185/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0435 - acc: 0.9875 - disc_loss: 1.3686 - disc_acc: 0.5450
Epoch 186/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0432 - acc: 0.9874 - disc_loss: 1.3683 - disc_acc: 0.5549
Epoch 187/500
266/266 [==============================] 

266/266 [==============================] - 1s 3ms/step - loss: 0.0374 - acc: 0.9898 - disc_loss: 1.3643 - disc_acc: 0.5468
Epoch 240/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0531 - acc: 0.9861 - disc_loss: 1.3659 - disc_acc: 0.5604
Epoch 241/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0401 - acc: 0.9899 - disc_loss: 1.3660 - disc_acc: 0.5557
Epoch 242/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0465 - acc: 0.9889 - disc_loss: 1.3699 - disc_acc: 0.5311
Epoch 243/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0403 - acc: 0.9884 - disc_loss: 1.3777 - disc_acc: 0.5345
Epoch 244/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0351 - acc: 0.9909 - disc_loss: 1.3763 - disc_acc: 0.5226
Epoch 245/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0378 - acc: 0.9906 - disc_loss: 1.3708 - disc_acc: 0.5357
Epoch 246/500
266/266 [==============================] 

266/266 [==============================] - 1s 3ms/step - loss: 0.0533 - acc: 0.9848 - disc_loss: 1.3775 - disc_acc: 0.5297
Epoch 299/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0680 - acc: 0.9811 - disc_loss: 1.3694 - disc_acc: 0.5514
Epoch 300/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0605 - acc: 0.9820 - disc_loss: 1.3621 - disc_acc: 0.5682
Epoch 301/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0395 - acc: 0.9899 - disc_loss: 1.3713 - disc_acc: 0.5464
Epoch 302/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0371 - acc: 0.9894 - disc_loss: 1.3658 - disc_acc: 0.5421
Epoch 303/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0605 - acc: 0.9824 - disc_loss: 1.3612 - disc_acc: 0.5552
Epoch 304/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0526 - acc: 0.9849 - disc_loss: 1.3632 - disc_acc: 0.5421
Epoch 305/500
266/266 [==============================] 

266/266 [==============================] - 1s 3ms/step - loss: 0.0542 - acc: 0.9848 - disc_loss: 1.3664 - disc_acc: 0.5404
Epoch 358/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0342 - acc: 0.9912 - disc_loss: 1.3775 - disc_acc: 0.5211
Epoch 359/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0365 - acc: 0.9895 - disc_loss: 1.3778 - disc_acc: 0.5305
Epoch 360/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0373 - acc: 0.9899 - disc_loss: 1.3766 - disc_acc: 0.5243
Epoch 361/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0313 - acc: 0.9928 - disc_loss: 1.3744 - disc_acc: 0.5256
Epoch 362/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0421 - acc: 0.9888 - disc_loss: 1.3692 - disc_acc: 0.5385
Epoch 363/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0465 - acc: 0.9858 - disc_loss: 1.3631 - disc_acc: 0.5484
Epoch 364/500
266/266 [==============================] 

266/266 [==============================] - 1s 3ms/step - loss: 0.0534 - acc: 0.9847 - disc_loss: 1.3573 - disc_acc: 0.5531
Epoch 417/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0511 - acc: 0.9839 - disc_loss: 1.3577 - disc_acc: 0.5574
Epoch 418/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0347 - acc: 0.9906 - disc_loss: 1.3582 - disc_acc: 0.5639
Epoch 419/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0276 - acc: 0.9922 - disc_loss: 1.3573 - disc_acc: 0.5478
Epoch 420/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0327 - acc: 0.9906 - disc_loss: 1.3565 - disc_acc: 0.5568
Epoch 421/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0424 - acc: 0.9889 - disc_loss: 1.3704 - disc_acc: 0.5370
Epoch 422/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0353 - acc: 0.9893 - disc_loss: 1.3651 - disc_acc: 0.5288
Epoch 423/500
266/266 [==============================] 

266/266 [==============================] - 1s 3ms/step - loss: 0.0265 - acc: 0.9921 - disc_loss: 1.3673 - disc_acc: 0.5366
Epoch 476/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0310 - acc: 0.9920 - disc_loss: 1.3690 - disc_acc: 0.5175
Epoch 477/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0434 - acc: 0.9867 - disc_loss: 1.3694 - disc_acc: 0.5346
Epoch 478/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0314 - acc: 0.9909 - disc_loss: 1.3682 - disc_acc: 0.5442
Epoch 479/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0348 - acc: 0.9893 - disc_loss: 1.3600 - disc_acc: 0.5525
Epoch 480/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0426 - acc: 0.9882 - disc_loss: 1.3535 - disc_acc: 0.5662
Epoch 481/500
266/266 [==============================] - 1s 3ms/step - loss: 0.0429 - acc: 0.9884 - disc_loss: 1.3531 - disc_acc: 0.5512
Epoch 482/500
266/266 [==============================] 

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(db_images_t))
print(np.shape(db_labels_t))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(base_images, base_labels,db_images_t,db_labels_t, epochs=my_epoch,verbose=1,batch_size = 32)
db_model.score(base_images, base_labels)
db_model.score(db_images, db_labels)
db_model.save_weights(save_check_pt+ '/db')

(8446, 512)
(8446, 35)
(373, 512)
(373, 35)
Epoch 1/500
269/269 [==============================] - 2s 4ms/step - loss: 2.7362 - acc: 0.3113 - disc_loss: 1.4578 - disc_acc: 0.4676
Epoch 2/500
269/269 [==============================] - 1s 3ms/step - loss: 2.1485 - acc: 0.4942 - disc_loss: 1.3883 - disc_acc: 0.5029
Epoch 3/500
269/269 [==============================] - 1s 3ms/step - loss: 1.7300 - acc: 0.5944 - disc_loss: 1.3880 - disc_acc: 0.5092
Epoch 4/500
269/269 [==============================] - 1s 3ms/step - loss: 1.4831 - acc: 0.6305 - disc_loss: 1.3840 - disc_acc: 0.5143
Epoch 5/500
269/269 [==============================] - 1s 3ms/step - loss: 1.3565 - acc: 0.6484 - disc_loss: 1.3812 - disc_acc: 0.5382
Epoch 6/500
269/269 [==============================] - 1s 3ms/step - loss: 1.2243 - acc: 0.6727 - disc_loss: 1.3854 - disc_acc: 0.5179
Epoch 7/500
269/269 [==============================] - 1s 3ms/step - loss: 1.0776 - acc: 0.7014 - disc_loss: 1.3845 - disc_acc: 0.5151
Epoch 8/500

Epoch 61/500
269/269 [==============================] - 1s 3ms/step - loss: 0.1018 - acc: 0.9747 - disc_loss: 1.3807 - disc_acc: 0.5288
Epoch 62/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0974 - acc: 0.9765 - disc_loss: 1.3779 - disc_acc: 0.5415
Epoch 63/500
269/269 [==============================] - 1s 3ms/step - loss: 0.1008 - acc: 0.9741 - disc_loss: 1.3798 - disc_acc: 0.5282
Epoch 64/500
269/269 [==============================] - 1s 3ms/step - loss: 0.1017 - acc: 0.9748 - disc_loss: 1.3831 - disc_acc: 0.5174
Epoch 65/500
269/269 [==============================] - 1s 3ms/step - loss: 0.1053 - acc: 0.9720 - disc_loss: 1.3805 - disc_acc: 0.5241
Epoch 66/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0999 - acc: 0.9725 - disc_loss: 1.3801 - disc_acc: 0.5326
Epoch 67/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0984 - acc: 0.9759 - disc_loss: 1.3814 - disc_acc: 0.5274
Epoch 68/500
269/269 [==========================

269/269 [==============================] - 1s 3ms/step - loss: 0.0539 - acc: 0.9860 - disc_loss: 1.3823 - disc_acc: 0.5178
Epoch 122/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0799 - acc: 0.9770 - disc_loss: 1.3784 - disc_acc: 0.5390
Epoch 123/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0707 - acc: 0.9819 - disc_loss: 1.3786 - disc_acc: 0.5333
Epoch 124/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0484 - acc: 0.9869 - disc_loss: 1.3831 - disc_acc: 0.5231
Epoch 125/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0749 - acc: 0.9802 - disc_loss: 1.3786 - disc_acc: 0.5333
Epoch 126/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0694 - acc: 0.9801 - disc_loss: 1.3760 - disc_acc: 0.5470
Epoch 127/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0783 - acc: 0.9784 - disc_loss: 1.3734 - disc_acc: 0.5553
Epoch 128/500
269/269 [==============================] 

269/269 [==============================] - 1s 3ms/step - loss: 0.0443 - acc: 0.9880 - disc_loss: 1.3842 - disc_acc: 0.5142
Epoch 181/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0887 - acc: 0.9753 - disc_loss: 1.3740 - disc_acc: 0.5535
Epoch 182/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0827 - acc: 0.9775 - disc_loss: 1.3774 - disc_acc: 0.5365
Epoch 183/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0483 - acc: 0.9879 - disc_loss: 1.3824 - disc_acc: 0.5242
Epoch 184/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0426 - acc: 0.9886 - disc_loss: 1.3832 - disc_acc: 0.5255
Epoch 185/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0490 - acc: 0.9866 - disc_loss: 1.3784 - disc_acc: 0.5332
Epoch 186/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0380 - acc: 0.9902 - disc_loss: 1.3828 - disc_acc: 0.5139
Epoch 187/500
269/269 [==============================] 

269/269 [==============================] - 1s 3ms/step - loss: 0.0544 - acc: 0.9829 - disc_loss: 1.3786 - disc_acc: 0.5314
Epoch 240/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0403 - acc: 0.9886 - disc_loss: 1.3797 - disc_acc: 0.5219
Epoch 241/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0415 - acc: 0.9876 - disc_loss: 1.3804 - disc_acc: 0.5179
Epoch 242/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0550 - acc: 0.9857 - disc_loss: 1.3770 - disc_acc: 0.5260
Epoch 243/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0536 - acc: 0.9853 - disc_loss: 1.3851 - disc_acc: 0.5123
Epoch 244/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0560 - acc: 0.9824 - disc_loss: 1.3825 - disc_acc: 0.5171
Epoch 245/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0436 - acc: 0.9866 - disc_loss: 1.3826 - disc_acc: 0.5213
Epoch 246/500
269/269 [==============================] 

269/269 [==============================] - 1s 3ms/step - loss: 0.0404 - acc: 0.9901 - disc_loss: 1.3819 - disc_acc: 0.5232
Epoch 299/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0523 - acc: 0.9854 - disc_loss: 1.3848 - disc_acc: 0.5121
Epoch 300/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0569 - acc: 0.9841 - disc_loss: 1.3782 - disc_acc: 0.5241
Epoch 301/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0443 - acc: 0.9874 - disc_loss: 1.3776 - disc_acc: 0.5415
Epoch 302/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0348 - acc: 0.9892 - disc_loss: 1.3791 - disc_acc: 0.5210
Epoch 303/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0421 - acc: 0.9878 - disc_loss: 1.3822 - disc_acc: 0.5201
Epoch 304/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0416 - acc: 0.9889 - disc_loss: 1.3843 - disc_acc: 0.5193
Epoch 305/500
269/269 [==============================] 

269/269 [==============================] - 1s 3ms/step - loss: 0.0411 - acc: 0.9878 - disc_loss: 1.3783 - disc_acc: 0.5384
Epoch 358/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0386 - acc: 0.9906 - disc_loss: 1.3791 - disc_acc: 0.5271
Epoch 359/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0313 - acc: 0.9920 - disc_loss: 1.3798 - disc_acc: 0.5226
Epoch 360/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0318 - acc: 0.9918 - disc_loss: 1.3776 - disc_acc: 0.5266
Epoch 361/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0481 - acc: 0.9865 - disc_loss: 1.3738 - disc_acc: 0.5325
Epoch 362/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0410 - acc: 0.9895 - disc_loss: 1.3737 - disc_acc: 0.5405
Epoch 363/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0486 - acc: 0.9854 - disc_loss: 1.3706 - disc_acc: 0.5448
Epoch 364/500
269/269 [==============================] 

269/269 [==============================] - 1s 3ms/step - loss: 0.0389 - acc: 0.9886 - disc_loss: 1.3826 - disc_acc: 0.5075
Epoch 417/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0243 - acc: 0.9939 - disc_loss: 1.3840 - disc_acc: 0.5013
Epoch 418/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0494 - acc: 0.9848 - disc_loss: 1.3819 - disc_acc: 0.5156
Epoch 419/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0747 - acc: 0.9780 - disc_loss: 1.3744 - disc_acc: 0.5425
Epoch 420/500
269/269 [==============================] - 1s 3ms/step - loss: 0.1090 - acc: 0.9690 - disc_loss: 1.3650 - disc_acc: 0.5553
Epoch 421/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0430 - acc: 0.9875 - disc_loss: 1.3786 - disc_acc: 0.5240
Epoch 422/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0331 - acc: 0.9925 - disc_loss: 1.3844 - disc_acc: 0.5211
Epoch 423/500
269/269 [==============================] 

269/269 [==============================] - 1s 3ms/step - loss: 0.0329 - acc: 0.9901 - disc_loss: 1.3807 - disc_acc: 0.5189
Epoch 476/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0440 - acc: 0.9875 - disc_loss: 1.3784 - disc_acc: 0.5242
Epoch 477/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0438 - acc: 0.9879 - disc_loss: 1.3773 - disc_acc: 0.5281
Epoch 478/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0372 - acc: 0.9887 - disc_loss: 1.3797 - disc_acc: 0.5232
Epoch 479/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0706 - acc: 0.9781 - disc_loss: 1.3763 - disc_acc: 0.5324
Epoch 480/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0620 - acc: 0.9816 - disc_loss: 1.3733 - disc_acc: 0.5361
Epoch 481/500
269/269 [==============================] - 1s 3ms/step - loss: 0.0315 - acc: 0.9913 - disc_loss: 1.3794 - disc_acc: 0.5208
Epoch 482/500
269/269 [==============================] 

In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(fn_images_t))
print(np.shape(fn_labels_t))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(base_images, base_labels,fn_images_t,fn_labels_t, epochs=my_epoch,verbose=1,batch_size = 32)
fn_model.score(base_images, base_labels)
fn_model.score(fn_images, fn_labels)
fn_model.save_weights(save_check_pt+ '/fn')

(8446, 512)
(8446, 35)
(48, 512)
(48, 35)
Epoch 1/500
264/264 [==============================] - 2s 4ms/step - loss: 2.7912 - acc: 0.2987 - disc_loss: 1.4319 - disc_acc: 0.4843
Epoch 2/500
264/264 [==============================] - 1s 4ms/step - loss: 2.0733 - acc: 0.5192 - disc_loss: 1.3741 - disc_acc: 0.5439
Epoch 3/500
264/264 [==============================] - 1s 3ms/step - loss: 1.6346 - acc: 0.6248 - disc_loss: 1.3572 - disc_acc: 0.5804
Epoch 4/500
264/264 [==============================] - 1s 3ms/step - loss: 1.3744 - acc: 0.6541 - disc_loss: 1.3583 - disc_acc: 0.5655
Epoch 5/500
264/264 [==============================] - 1s 3ms/step - loss: 1.2039 - acc: 0.6812 - disc_loss: 1.3502 - disc_acc: 0.5801
Epoch 6/500
264/264 [==============================] - 1s 3ms/step - loss: 1.0667 - acc: 0.7088 - disc_loss: 1.3450 - disc_acc: 0.5778
Epoch 7/500
264/264 [==============================] - 1s 3ms/step - loss: 0.9478 - acc: 0.7391 - disc_loss: 1.3492 - disc_acc: 0.5719
Epoch 8/500
2

Epoch 61/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1382 - acc: 0.9654 - disc_loss: 1.3444 - disc_acc: 0.5802
Epoch 62/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1648 - acc: 0.9570 - disc_loss: 1.3401 - disc_acc: 0.5848
Epoch 63/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1551 - acc: 0.9619 - disc_loss: 1.3397 - disc_acc: 0.5893
Epoch 64/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1438 - acc: 0.9599 - disc_loss: 1.3593 - disc_acc: 0.5644
Epoch 65/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1504 - acc: 0.9593 - disc_loss: 1.3374 - disc_acc: 0.5985
Epoch 66/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1170 - acc: 0.9725 - disc_loss: 1.3477 - disc_acc: 0.5887
Epoch 67/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1268 - acc: 0.9699 - disc_loss: 1.3395 - disc_acc: 0.5916
Epoch 68/500
264/264 [==========================

264/264 [==============================] - 1s 3ms/step - loss: 0.1045 - acc: 0.9728 - disc_loss: 1.3365 - disc_acc: 0.5986
Epoch 122/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0947 - acc: 0.9764 - disc_loss: 1.3343 - disc_acc: 0.5881
Epoch 123/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1156 - acc: 0.9679 - disc_loss: 1.3474 - disc_acc: 0.5734
Epoch 124/500
264/264 [==============================] - 1s 4ms/step - loss: 0.1149 - acc: 0.9699 - disc_loss: 1.3531 - disc_acc: 0.5752
Epoch 125/500
264/264 [==============================] - 1s 4ms/step - loss: 0.1190 - acc: 0.9690 - disc_loss: 1.3589 - disc_acc: 0.5715
Epoch 126/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0997 - acc: 0.9732 - disc_loss: 1.3593 - disc_acc: 0.5728
Epoch 127/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0993 - acc: 0.9744 - disc_loss: 1.3510 - disc_acc: 0.5680
Epoch 128/500
264/264 [==============================] 

264/264 [==============================] - 1s 3ms/step - loss: 0.0829 - acc: 0.9786 - disc_loss: 1.3470 - disc_acc: 0.5923
Epoch 181/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1193 - acc: 0.9665 - disc_loss: 1.3514 - disc_acc: 0.5848
Epoch 182/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0853 - acc: 0.9786 - disc_loss: 1.3512 - disc_acc: 0.5779
Epoch 183/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0745 - acc: 0.9792 - disc_loss: 1.3379 - disc_acc: 0.5909
Epoch 184/500
264/264 [==============================] - 1s 4ms/step - loss: 0.0767 - acc: 0.9793 - disc_loss: 1.3334 - disc_acc: 0.5903
Epoch 185/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0559 - acc: 0.9864 - disc_loss: 1.3279 - disc_acc: 0.5958
Epoch 186/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0642 - acc: 0.9838 - disc_loss: 1.3300 - disc_acc: 0.6078
Epoch 187/500
264/264 [==============================] 

264/264 [==============================] - 1s 3ms/step - loss: 0.0852 - acc: 0.9766 - disc_loss: 1.3361 - disc_acc: 0.5832
Epoch 240/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0805 - acc: 0.9785 - disc_loss: 1.3145 - disc_acc: 0.6027
Epoch 241/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0859 - acc: 0.9755 - disc_loss: 1.3255 - disc_acc: 0.5927
Epoch 242/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0798 - acc: 0.9776 - disc_loss: 1.3363 - disc_acc: 0.5833
Epoch 243/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1000 - acc: 0.9718 - disc_loss: 1.3326 - disc_acc: 0.5970
Epoch 244/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1024 - acc: 0.9724 - disc_loss: 1.3206 - disc_acc: 0.5962
Epoch 245/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0927 - acc: 0.9741 - disc_loss: 1.3444 - disc_acc: 0.5887
Epoch 246/500
264/264 [==============================] 

264/264 [==============================] - 1s 3ms/step - loss: 0.0814 - acc: 0.9788 - disc_loss: 1.3332 - disc_acc: 0.5810
Epoch 299/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0800 - acc: 0.9767 - disc_loss: 1.3328 - disc_acc: 0.6038
Epoch 300/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0759 - acc: 0.9770 - disc_loss: 1.3226 - disc_acc: 0.5992
Epoch 301/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0838 - acc: 0.9763 - disc_loss: 1.3418 - disc_acc: 0.5819
Epoch 302/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1119 - acc: 0.9671 - disc_loss: 1.3440 - disc_acc: 0.5858
Epoch 303/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1298 - acc: 0.9632 - disc_loss: 1.3255 - disc_acc: 0.6005
Epoch 304/500
264/264 [==============================] - 1s 4ms/step - loss: 0.0833 - acc: 0.9785 - disc_loss: 1.3435 - disc_acc: 0.6047
Epoch 305/500
264/264 [==============================] 

264/264 [==============================] - 1s 3ms/step - loss: 0.0858 - acc: 0.9747 - disc_loss: 1.3447 - disc_acc: 0.5716
Epoch 358/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0859 - acc: 0.9770 - disc_loss: 1.3567 - disc_acc: 0.5563
Epoch 359/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0873 - acc: 0.9764 - disc_loss: 1.3464 - disc_acc: 0.5747
Epoch 360/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0670 - acc: 0.9821 - disc_loss: 1.3386 - disc_acc: 0.5827
Epoch 361/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0780 - acc: 0.9779 - disc_loss: 1.3431 - disc_acc: 0.5792
Epoch 362/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0792 - acc: 0.9768 - disc_loss: 1.3441 - disc_acc: 0.5732
Epoch 363/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0933 - acc: 0.9760 - disc_loss: 1.3519 - disc_acc: 0.5523
Epoch 364/500
264/264 [==============================] 

264/264 [==============================] - 1s 3ms/step - loss: 0.0819 - acc: 0.9772 - disc_loss: 1.3217 - disc_acc: 0.6146
Epoch 417/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0890 - acc: 0.9721 - disc_loss: 1.3480 - disc_acc: 0.5734
Epoch 418/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0672 - acc: 0.9812 - disc_loss: 1.3208 - disc_acc: 0.5991
Epoch 419/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0876 - acc: 0.9748 - disc_loss: 1.3178 - disc_acc: 0.6115
Epoch 420/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0808 - acc: 0.9763 - disc_loss: 1.3141 - disc_acc: 0.6157
Epoch 421/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0960 - acc: 0.9723 - disc_loss: 1.3244 - disc_acc: 0.5920
Epoch 422/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1121 - acc: 0.9670 - disc_loss: 1.3219 - disc_acc: 0.5996
Epoch 423/500
264/264 [==============================] 

264/264 [==============================] - 1s 3ms/step - loss: 0.0728 - acc: 0.9799 - disc_loss: 1.3388 - disc_acc: 0.5770
Epoch 476/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0634 - acc: 0.9840 - disc_loss: 1.3384 - disc_acc: 0.5836
Epoch 477/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0797 - acc: 0.9781 - disc_loss: 1.3445 - disc_acc: 0.5654
Epoch 478/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0788 - acc: 0.9780 - disc_loss: 1.3350 - disc_acc: 0.5802
Epoch 479/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0717 - acc: 0.9818 - disc_loss: 1.3320 - disc_acc: 0.5835
Epoch 480/500
264/264 [==============================] - 1s 3ms/step - loss: 0.0851 - acc: 0.9787 - disc_loss: 1.3236 - disc_acc: 0.5854
Epoch 481/500
264/264 [==============================] - 1s 3ms/step - loss: 0.1114 - acc: 0.9693 - disc_loss: 1.3287 - disc_acc: 0.5775
Epoch 482/500
264/264 [==============================] 